In [1]:
import tensorflow as tf
 
# Display the version
print(tf.__version__)    

# other imports
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, Add, ReLU
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model

2023-04-09 13:41:27.538507: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.12.0


In [2]:
# Load in the data
cifar10 = tf.keras.datasets.cifar10
 
# Distribute it to train and test set
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


In [3]:
# Reduce pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0
 
# flatten the label values
y_train, y_test = y_train.flatten(), y_test.flatten()

In [ ]:
'''
Opción 1:
tf.keras.applications.ResNet50(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=avg,
    classes=10,
    classifier_activation='softmax'
)

https://keras.io/api/applications/resnet/
'''

In [ ]:
# Opción 2: hacerla a manito. Pero haré resnet 34.

def resblock(x, filters:int, strides:tuple):
    conv1 = Conv2D(filters=filters, kernel_size=(3,3), strides=strides, padding="same")(x)
    r = ReLU()(conv1)
    conv2 = Conv2D(filters=filters, kernel_size=(3,3), strides=strides, padding="same" )(conv1)
    add = Add()([x,conv2])
    r2 = ReLU()(add)
    return r2